In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
print(module_path)

/Users/revilooliver/Documents/quantum_computing/research/pauli_sandwitching/cut4mitigation


In [3]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy
import circuit_cutter
import mlrecon_methods as ml
import numpy as np
import mthree

import qiskit
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

import qiskit.providers.aer.noise as noise
from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, total_counts, norm_dict
from utils.vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation, apply_checking_circuit
from mlrecon_methods import run_circuits, collect_fragment_circuits

In [4]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.22.2', 'qiskit-aer': '0.11.1', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.19.2', 'qiskit': '0.39.2', 'qiskit-nature': '0.5.1', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [5]:
numpy.set_printoptions(linewidth = 200)

simulation_backend = "qasm_simulator"

In [6]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-ornl', group='ornl', project='chm185')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_montreal') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_kolkata') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_mumbai') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_guadalupe') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-ornl', grou

In [7]:
device = provider.get_backend('ibm_auckland')
device

<IBMQBackend('ibm_auckland') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>

In [8]:
device.configuration().to_dict()['max_shots']

100000

In [9]:
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
noise_model = NoiseModel.from_backend(device, gate_error = False, thermal_relaxation= False)
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('measure', (4,)), ('measure', (5,)), ('measure', (6,)), ('measure', (7,)), ('measure', (8,)), ('measure', (9,)), ('measure', (10,)), ('measure', (11,)), ('measure', (12,)), ('measure', (13,)), ('measure', (14,)), ('measure', (15,)), ('measure', (16,)), ('measure', (17,)), ('measure', (18,)), ('measure', (19,)), ('measure', (20,)), ('measure', (21,)), ('measure', (22,)), ('measure', (23,)), ('measure', (24,)), ('measure', (25,)), ('measure', (26,))]


In [10]:
noisy_simulator = provider.get_backend('ibmq_qasm_simulator')

In [11]:
front_layer = QuantumCircuit.from_qasm_file("qasm/vqe_LiH_front.qasm")

In [12]:
mid_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_mid.qasm")

In [13]:
end_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_end.qasm")

In [14]:
orign_circuit = front_layer + mid_layer + end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_23476/743480212.py:1: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  orign_circuit = front_layer + mid_layer + end_layer


In [15]:
orign_circuit.draw()

┌───┐       ┌───────────────┐┌───────────────┐   ┌───────────────┐»
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Ry(0.0082611) ├»
            ├───┤       └┬──────────────┤├───────────────┤ │ └───────────────┘»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■─────────■────────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘           │        »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────────────────────────■────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                                     »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├──────────────────────────────────────»
     └─────────────────┘  └────────────┘                                      »
q_4: ─────────────────────────────────────────────────────────────────────────»
                                                                              »
«      ┌────────────┐                                    
«q_0: ─┤ Rz(1.3184) ├────────────────────────────────────
«     ┌┴────────────┴┐   ┌────────────┐                  
«q_1: ┤ Ry(0.013556) ├───┤ Rz(1.3353) ├──────────────────
«     └──────────────┘  ┌┴────────────┴┐ ┌──────────────┐
«q_2: ───────■──────────┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├
«            │        ┌─┴──────────────┴┐└┬────────────┬┘
«q_3: ───────■────────┤ Ry(-4.5556e-05) ├─┤ Rz(1.2199) ├─
«                     └─────────────────┘ └────────────┘ 
«q_4: ───────────────────────────────────────────────────
«

In [16]:
#checking circuit

In [17]:
phase_y_front = [0.008239051732521298, 0.013635804359443573, 0.05162548888524782, -2.1999413989625588e-05]
phase_z_front = [-0.010143905866716232, -0.028142425222093263, 0.008659821134257232, 1.4378429162081177]
phase_y_end = [0.008261085257040952, 0.013555948647231175, 0.0516520269898158, -4.555585169085397e-05]
phase_z_end = [1.3183908530236275, 1.3352666291100785, -0.26460520551963923, 1.2198754282688027]
x_front = [True, True, False, False]

In [19]:
qubit_id = 1
circuit = QuantumCircuit(5)
apply_checking_circuit(circuit, [qubit_id], [4], side = 'front', phase_y = phase_y_front[qubit_id], phase_z = phase_z_front[qubit_id], x = x_front[qubit_id])
circuit += orign_circuit
apply_checking_circuit(circuit, [qubit_id], [4], side = 'end', phase_y = phase_y_end[qubit_id], phase_z = phase_z_end[qubit_id])

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_23476/911978416.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += orign_circuit


In [20]:
circuit.draw()

┌───┐       ┌───────────────┐┌───────────────┐                   »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├───────────────────»
            ├───┤       └┬──────────────┤├───────────────┤   ┌──────────────┐»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■─┤ Rz(0.028142) ├»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘ │ └──────────────┘»
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├──────────────────┼─────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                  │                 »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼─────────────────»
     └──────┬───┬──────┘  └────────────┘                   │                 »
q_4: ───────┤ H ├──────────────────────────────────────────■─────────────────»
            └───┘                                                            »
«                                                                    »
«q_0: ─────────────────────────────────────────────────────────────■─»
«     ┌───────────────┐┌───┐┌───┐┌──────────────┐┌───────────────┐ │ »
«q_1: ┤ Ry(-0.013636) ├┤ X ├┤ X ├┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■─»
«     └───────────────┘└───┘└───┘└──────────────┘└───────────────┘   »
«q_2: ───────────────────────────────────────────────────────────────»
«                                                                    »
«q_3: ───────────────────────────────────────────────────────────────»
«                                                                    »
«q_4: ───────────────────────────────────────────────────────────────»
«                                                                    »
«     ┌───────────────┐ ┌────────────┐                                    »
«q_0: ┤ Ry(0.0082611) ├─┤ Rz(1.3184) ├────────────────────────────────────»
«     └───────────────┘┌┴────────────┴┐   ┌────────────┐  ┌─────────────┐ »
«q_1: ────────■────────┤ Ry(0.013556) ├───┤ Rz(1.3353) ├──┤ Rz(-1.3353) ├─»
«             │        └──────────────┘  ┌┴────────────┴┐ ├─────────────┴┐»
«q_2: ────────■───────────────■──────────┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├»
«                             │        ┌─┴──────────────┴┐└┬────────────┬┘»
«q_3: ────────────────────────■────────┤ Ry(-4.5556e-05) ├─┤ Rz(1.2199) ├─»
«                                      └─────────────────┘ └────────────┘ »
«q_4: ────────────────────────────────────────────────────────────────────»
«                                                                         »
«                                                       
«q_0: ──────────────────────────────────────────────────
«     ┌───────────────┐   ┌──────────────┐┌────────────┐
«q_1: ┤ Ry(-0.013556) ├─■─┤ Ry(0.013556) ├┤ Rz(1.3353) ├
«     └───────────────┘ │ └──────────────┘└────────────┘
«q_2: ──────────────────┼───────────────────────────────
«                       │                               
«q_3: ──────────────────┼───────────────────────────────
«                       │      ┌───┐                    
«q_4: ──────────────────■──────┤ H ├────────────────────
«                              └───┘

In [21]:
pauli_list = read_from_file('LiH_hamiltonian_1.5.txt')
pauli_commute = find_commute_groups(pauli_list)

In [22]:
#generate individual measurment circuits that changes the basis
meas_qcs = []
for i in range(0, len(pauli_commute)):
    temp_qc = MeasureCircuit(pauli_commute[i], num_qubits = 4,num_qargs = 5)
    meas_qcs.append(temp_qc)
    print(temp_qc)

YYYY
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
XZYY
      ┌────────────┐ 
q_0: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_1: ────────────────
     ┌──────────────┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYXZ
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_2: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_3: ────────────────
                     
q_4: ────────────────
                     
XXYY
      ┌────────────┐ 
q_0: ─┤ U(π/2,0,π) ├─
      ├────────────┤ 
q_1: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYXX
     ┌─

In [23]:
circuit_list = []
for meas in meas_qcs:
    temp_circuit = circuit.copy()
    temp_circuit += meas
    circuit_list.append(temp_circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_23476/1717817760.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  temp_circuit += meas


In [24]:
circuit_list[-2].draw()

┌───┐       ┌───────────────┐┌───────────────┐                   »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├───────────────────»
            ├───┤       └┬──────────────┤├───────────────┤   ┌──────────────┐»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■─┤ Rz(0.028142) ├»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘ │ └──────────────┘»
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├──────────────────┼─────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                  │                 »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼─────────────────»
     └──────┬───┬──────┘  └────────────┘                   │                 »
q_4: ───────┤ H ├──────────────────────────────────────────■─────────────────»
            └───┘                                                            »
«                                                                    »
«q_0: ─────────────────────────────────────────────────────────────■─»
«     ┌───────────────┐┌───┐┌───┐┌──────────────┐┌───────────────┐ │ »
«q_1: ┤ Ry(-0.013636) ├┤ X ├┤ X ├┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■─»
«     └───────────────┘└───┘└───┘└──────────────┘└───────────────┘   »
«q_2: ───────────────────────────────────────────────────────────────»
«                                                                    »
«q_3: ───────────────────────────────────────────────────────────────»
«                                                                    »
«q_4: ───────────────────────────────────────────────────────────────»
«                                                                    »
«     ┌───────────────┐ ┌────────────┐                                    »
«q_0: ┤ Ry(0.0082611) ├─┤ Rz(1.3184) ├────────────────────────────────────»
«     └───────────────┘┌┴────────────┴┐   ┌────────────┐  ┌─────────────┐ »
«q_1: ────────■────────┤ Ry(0.013556) ├───┤ Rz(1.3353) ├──┤ Rz(-1.3353) ├─»
«             │        └──────────────┘  ┌┴────────────┴┐ ├─────────────┴┐»
«q_2: ────────■───────────────■──────────┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├»
«                             │        ┌─┴──────────────┴┐└┬────────────┬┘»
«q_3: ────────────────────────■────────┤ Ry(-4.5556e-05) ├─┤ Rz(1.2199) ├─»
«                                      └─────────────────┘ └────────────┘ »
«q_4: ────────────────────────────────────────────────────────────────────»
«                                                                         »
«                                                       
«q_0: ──────────────────────────────────────────────────
«     ┌───────────────┐   ┌──────────────┐┌────────────┐
«q_1: ┤ Ry(-0.013556) ├─■─┤ Ry(0.013556) ├┤ Rz(1.3353) ├
«     └───────────────┘ │ └──────────────┘└────────────┘
«q_2: ──────────────────┼───────────────────────────────
«       ┌────────────┐  │                               
«q_3: ──┤ U(π/2,0,π) ├──┼───────────────────────────────
«       └────────────┘  │      ┌───┐                    
«q_4: ──────────────────■──────┤ H ├────────────────────
«                              └───┘

In [25]:
# cuts = []
# for i in range(0, 1):
#     if i == 0 or i == qubits - 1:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 6)]
#     else:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 4)]
#     cuts += cut

In [26]:
cuts = [(circuit.qubits[1], 7),(circuit.qubits[1], 14)]

In [27]:
fragments_list = []
wire_path_map_list = []
for temp_qc in circuit_list:
    print(temp_qc)
    fragments, wire_path_map = circuit_cutter.cut_circuit(temp_qc, cuts)
    fragments_list.append(fragments)
    wire_path_map_list.append(wire_path_map)

            ┌───┐       ┌───────────────┐┌───────────────┐                   »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├───────────────────»
            ├───┤       └┬──────────────┤├───────────────┤   ┌──────────────┐»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■─┤ Rz(0.028142) ├»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘ │ └──────────────┘»
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├──────────────────┼─────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                  │                 »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼─────────────────»
     └──────┬───┬──────┘  └────────────┘                   │                 »
q_4: ───────┤ H ├──────────────────────────────────────────■─────────────────»
            └───┘                                                            »
«                                                                    »
«q_0: ──────────────────────────────────────────────────────

            ┌───┐       ┌───────────────┐┌───────────────┐                   »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├───────────────────»
            ├───┤       └┬──────────────┤├───────────────┤   ┌──────────────┐»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■─┤ Rz(0.028142) ├»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘ │ └──────────────┘»
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├──────────────────┼─────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                  │                 »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼─────────────────»
     └──────┬───┬──────┘  └────────────┘                   │                 »
q_4: ───────┤ H ├──────────────────────────────────────────■─────────────────»
            └───┘                                                            »
«                                                                    »
«q_0: ──────────────────────────────────────────────────────

            ┌───┐       ┌───────────────┐┌───────────────┐                   »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├───────────────────»
            ├───┤       └┬──────────────┤├───────────────┤   ┌──────────────┐»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■─┤ Rz(0.028142) ├»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘ │ └──────────────┘»
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├──────────────────┼─────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                  │                 »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼─────────────────»
     └──────┬───┬──────┘  └────────────┘                   │                 »
q_4: ───────┤ H ├──────────────────────────────────────────■─────────────────»
            └───┘                                                            »
«                                                                    »
«q_0: ──────────────────────────────────────────────────────

            ┌───┐       ┌───────────────┐┌───────────────┐                   »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├───────────────────»
            ├───┤       └┬──────────────┤├───────────────┤   ┌──────────────┐»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■─┤ Rz(0.028142) ├»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘ │ └──────────────┘»
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├──────────────────┼─────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                  │                 »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼─────────────────»
     └──────┬───┬──────┘  └────────────┘                   │                 »
q_4: ───────┤ H ├──────────────────────────────────────────■─────────────────»
            └───┘                                                            »
«                                                                    »
«q_0: ──────────────────────────────────────────────────────

            ┌───┐       ┌───────────────┐┌───────────────┐                   »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├───────────────────»
            ├───┤       └┬──────────────┤├───────────────┤   ┌──────────────┐»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■─┤ Rz(0.028142) ├»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘ │ └──────────────┘»
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├──────────────────┼─────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                  │                 »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼─────────────────»
     └──────┬───┬──────┘  └────────────┘                   │                 »
q_4: ───────┤ H ├──────────────────────────────────────────■─────────────────»
            └───┘                                                            »
«                                                                    »
«q_0: ──────────────────────────────────────────────────────

            ┌───┐       ┌───────────────┐┌───────────────┐                   »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├───────────────────»
            ├───┤       └┬──────────────┤├───────────────┤   ┌──────────────┐»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■─┤ Rz(0.028142) ├»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘ │ └──────────────┘»
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├──────────────────┼─────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                  │                 »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼─────────────────»
     └──────┬───┬──────┘  └────────────┘                   │                 »
q_4: ───────┤ H ├──────────────────────────────────────────■─────────────────»
            └───┘                                                            »
«                                                                    »
«q_0: ──────────────────────────────────────────────────────

            ┌───┐       ┌───────────────┐┌───────────────┐                   »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├───────────────────»
            ├───┤       └┬──────────────┤├───────────────┤   ┌──────────────┐»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■─┤ Rz(0.028142) ├»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘ │ └──────────────┘»
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├──────────────────┼─────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                  │                 »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼─────────────────»
     └──────┬───┬──────┘  └────────────┘                   │                 »
q_4: ───────┤ H ├──────────────────────────────────────────■─────────────────»
            └───┘                                                            »
«                                                                    »
«q_0: ──────────────────────────────────────────────────────

In [28]:
print(wire_path_map_list[0])

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(4, 'q'), 2)),), Qubit(QuantumRegister(5, 'q'), 1): ((1, Qubit(QuantumRegister(3, 'q'), 0)), (0, Qubit(QuantumRegister(4, 'q'), 3)), (1, Qubit(QuantumRegister(3, 'q'), 2))), Qubit(QuantumRegister(5, 'q'), 2): ((0, Qubit(QuantumRegister(4, 'q'), 0)),), Qubit(QuantumRegister(5, 'q'), 3): ((0, Qubit(QuantumRegister(4, 'q'), 1)),), Qubit(QuantumRegister(5, 'q'), 4): ((1, Qubit(QuantumRegister(3, 'q'), 1)),)}


In [29]:
total_variants = ml.fragment_variants(wire_path_map_list[0])

In [30]:
total_variants

24

In [31]:
shots = 10000 * total_variants

In [32]:
fragments_list[0][0].draw()

┌──────────────┐ ┌───────────────┐                                     »
q_0: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├─────────────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                                     »
q_1: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├──────────────────────────────────────»
     └──────┬───┬──────┘┌─┴────────────┴┐┌───────────────┐   ┌───────────────┐»
q_2: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Ry(0.0082611) ├»
            ├───┤       └┬──────────────┤├───────────────┤ │ └───────────────┘»
q_3: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■──────────────────»
            └───┘        └──────────────┘└───────────────┘                    »
«                          ┌──────────────┐ ┌──────────────┐┌──────────────┐
«q_0: ─■────────■──────────┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├┤ U(π/2,0,π/2) ├
«      │        │        ┌─┴──────────────┴┐└┬────────────┬┘├──────────────┤
«q_1: ─┼────────■────────┤ Ry(-4.5556e-05) ├─┤ Rz(1.2199) ├─┤ U(π/2,0,π/2) ├
«      │  ┌────────────┐ └─┬──────────────┬┘ └────────────┘ └──────────────┘
«q_2: ─┼──┤ Rz(1.3184) ├───┤ U(π/2,0,π/2) ├─────────────────────────────────
«      │ ┌┴────────────┴┐  └┬────────────┬┘                                 
«q_3: ─■─┤ Ry(0.013556) ├───┤ Rz(1.3353) ├──────────────────────────────────
«        └──────────────┘   └────────────┘

In [33]:
fragment_cuts_list = []
for index in range(0, len(fragments_list)):
    for idx, fragment in enumerate(fragments_list[index]):
        print(f"fragment {idx}:")
        print(fragment)
        print()
    fragment_cuts = ml.fragment_cuts(wire_path_map_list[index])
    fragment_cuts_list.append(fragment_cuts)


fragment 0:
       ┌──────────────┐ ┌───────────────┐                                     »
q_0: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├─────────────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                                     »
q_1: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├──────────────────────────────────────»
     └──────┬───┬──────┘┌─┴────────────┴┐┌───────────────┐   ┌───────────────┐»
q_2: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Ry(0.0082611) ├»
            ├───┤       └┬──────────────┤├───────────────┤ │ └───────────────┘»
q_3: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■──────────────────»
            └───┘        └──────────────┘└───────────────┘                    »
«                          ┌──────────────┐ ┌──────────────┐┌──────────────┐
«q_0: ─■────────■──────────┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├┤ U(π/2,0,π/2) ├
«      │        │        ┌─┴──────────────┴┐└┬────────────┬┘├──────────────┤
«q_1: ─┼────────■────────┤ Ry(-4.5556

       ┌──────────────┐ ┌───────────────┐                                     »
q_0: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├─────────────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                                     »
q_1: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├──────────────────────────────────────»
     └──────┬───┬──────┘┌─┴────────────┴┐┌───────────────┐   ┌───────────────┐»
q_2: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Ry(0.0082611) ├»
            ├───┤       └┬──────────────┤├───────────────┤ │ └───────────────┘»
q_3: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■──────────────────»
            └───┘        └──────────────┘└───────────────┘                    »
«                          ┌──────────────┐ ┌──────────────┐
«q_0: ─■────────■──────────┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├
«      │        │        ┌─┴──────────────┴┐└┬────────────┬┘
«q_1: ─┼────────■────────┤ Ry(-4.5556e-05) ├─┤ Rz(1.2199) ├─
«      │  ┌────────────┐ └──────────

In [34]:
print(fragment_cuts_list[-1])

[{'prep': 1, 'meas': 1}, {'prep': 1, 'meas': 1}]


In [35]:
wire_path_map

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(4, 'q'), 2)),),
 Qubit(QuantumRegister(5, 'q'), 1): ((1, Qubit(QuantumRegister(3, 'q'), 0)),
  (0, Qubit(QuantumRegister(4, 'q'), 3)),
  (1, Qubit(QuantumRegister(3, 'q'), 2))),
 Qubit(QuantumRegister(5, 'q'), 2): ((0, Qubit(QuantumRegister(4, 'q'), 0)),),
 Qubit(QuantumRegister(5, 'q'), 3): ((0, Qubit(QuantumRegister(4, 'q'), 1)),),
 Qubit(QuantumRegister(5, 'q'), 4): ((1, Qubit(QuantumRegister(3, 'q'), 1)),)}

In [36]:
num_fragments = len(fragments)
num_fragments

2

In [37]:
num_tomo_circuits = len(meas_qcs)
num_tomo_circuits

25

In [38]:
fragments[0].draw()

┌──────────────┐ ┌───────────────┐                                     »
q_0: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├─────────────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                                     »
q_1: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├──────────────────────────────────────»
     └──────┬───┬──────┘┌─┴────────────┴┐┌───────────────┐   ┌───────────────┐»
q_2: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Ry(0.0082611) ├»
            ├───┤       └┬──────────────┤├───────────────┤ │ └───────────────┘»
q_3: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■──────────────────»
            └───┘        └──────────────┘└───────────────┘                    »
«                          ┌──────────────┐ ┌──────────────┐
«q_0: ─■────────■──────────┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├
«      │        │        ┌─┴──────────────┴┐└┬────────────┬┘
«q_1: ─┼────────■────────┤ Ry(-4.5556e-05) ├─┤ Rz(1.2199) ├─
«      │  ┌────────────┐ └─────────────────┘ └────────────┘ 
«q_2: ─┼──┤ Rz(1.3184) ├────────────────────────────────────
«      │ ┌┴────────────┴┐   ┌────────────┐                  
«q_3: ─■─┤ Ry(0.013556) ├───┤ Rz(1.3353) ├──────────────────
«        └──────────────┘   └────────────┘

In [39]:
fragments[1].draw()

┌───┐      ┌──────────────┐┌───────────────┐   ┌──────────────┐»
q_0: ─────┤ X ├──────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■─┤ Rz(0.028142) ├»
          ├───┤      └──────────────┘└───────────────┘ │ └──────────────┘»
q_1: ─────┤ H ├────────────────────────────────────────■────────■────────»
     ┌────┴───┴────┐┌───────────────┐                           │        »
q_2: ┤ Rz(-1.3353) ├┤ Ry(-0.013556) ├───────────────────────────■────────»
     └─────────────┘└───────────────┘                                    »
«     ┌───────────────┐    ┌───┐     
«q_0: ┤ Ry(-0.013636) ├────┤ X ├─────
«     └─────┬───┬─────┘    └───┘     
«q_1: ──────┤ H ├────────────────────
«      ┌────┴───┴─────┐┌────────────┐
«q_2: ─┤ Ry(0.013556) ├┤ Rz(1.3353) ├
«      └──────────────┘└────────────┘

In [41]:
hardware_index = 0

In [42]:
print(shots)

240000


In [43]:
tomography_circuits = []
for idx in range(0, len(fragments_list)):
    fragments = fragments_list[idx]
    wire_path_map = wire_path_map_list[idx]
    temp_frag_circuits = [0] * len(fragments)
    for i in range(0, len(fragments)):
        if i == hardware_index:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
        else:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
    tomography_circuits.append(temp_frag_circuits)

In [44]:
#contains 34 basis
len(tomography_circuits)

25

In [45]:
#contains two fragments
len(tomography_circuits[0])

2

In [46]:
len(tomography_circuits[0][1])

1

In [47]:
#tomography circuits
len(tomography_circuits[0][1][0])

12

In [48]:
multi_exp_circuits = [[],[]]
for i in range(0, len(tomography_circuits)):
    #for each basis collect the three fragments:
    for j in range(0, num_fragments):
        fragment_circ = tomography_circuits[i][j][0]
        multi_exp_circuits[j] += fragment_circ

In [49]:
#16,14,11,8
#[11,8,16,14]

In [51]:
multi_exp_circuits[hardware_index][-200].draw()

░   ┌──────────────┐ ┌───────────────┐»
  q_0: ────────────────────░───┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├»
                           ░ ┌─┴──────────────┴┐└─┬────────────┬┘»
  q_1: ────────────────────░─┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├─»
                           ░ └──────┬───┬──────┘┌─┴────────────┴┐»
  q_2: ────────────────────░────────┤ X ├───────┤ Ry(0.0082391) ├»
       ┌─────────────────┐ ░        ├───┤       └┬──────────────┤»
  q_3: ┤ U3(-1.9106,π,0) ├─░────────┤ X ├────────┤ Ry(0.013636) ├»
       └─────────────────┘ ░        └───┘        └──────────────┘»
c16_0: ══════════════════════════════════════════════════════════»
                                                                 »
c16_1: ══════════════════════════════════════════════════════════»
                                                                 »
c16_2: ══════════════════════════════════════════════════════════»
                                                                 »
c16_3: ══════════════════════════════════════════════════════════»
                                                                 »
«                                                               »
«  q_0: ──────────────────────────────────────■────────■────────»
«                                             │        │        »
«  q_1: ──────────────────────────────────────┼────────■────────»
«       ┌───────────────┐   ┌───────────────┐ │  ┌────────────┐ »
«  q_2: ┤ Rz(-0.010144) ├─■─┤ Ry(0.0082611) ├─┼──┤ Rz(1.3184) ├─»
«       ├───────────────┤ │ └───────────────┘ │ ┌┴────────────┴┐»
«  q_3: ┤ Rz(-0.028142) ├─■───────────────────■─┤ Ry(0.013556) ├»
«       └───────────────┘                       └──────────────┘»
«c16_0: ════════════════════════════════════════════════════════»
«                                                               »
«c16_1: ════════════════════════════════════════════════════════»
«                                                               »
«c16_2: ════════════════════════════════════════════════════════»
«                                                               »
«c16_3: ════════════════════════════════════════════════════════»
«                                                               »
«         ┌──────────────┐ ┌──────────────┐┌──────────────┐ ░ ┌─┐              
«  q_0: ──┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├┤ U(π/2,0,π/2) ├─░─┤M├──────────────
«       ┌─┴──────────────┴┐└┬────────────┬┘├──────────────┤ ░ └╥┘┌─┐           
«  q_1: ┤ Ry(-4.5556e-05) ├─┤ Rz(1.2199) ├─┤ U(π/2,0,π/2) ├─░──╫─┤M├───────────
«       └─────────────────┘ └────────────┘ └──────────────┘ ░  ║ └╥┘┌─┐        
«  q_2: ────────────────────────────────────────────────────░──╫──╫─┤M├────────
«          ┌────────────┐                                   ░  ║  ║ └╥┘┌───┐┌─┐
«  q_3: ───┤ Rz(1.3353) ├───────────────────────────────────░──╫──╫──╫─┤ H ├┤M├
«          └────────────┘                                   ░  ║  ║  ║ └───┘└╥┘
«c16_0: ═══════════════════════════════════════════════════════╩══╬══╬═══════╬═
«                                                                 ║  ║       ║ 
«c16_1: ══════════════════════════════════════════════════════════╩══╬═══════╬═
«                                                                    ║       ║ 
«c16_2: ═════════════════════════════════════════════════════════════╩═══════╬═
«                                                                            ║ 
«c16_3: ═════════════════════════════════════════════════════════════════════╩═
«

In [78]:
frag_datas = [0] * num_fragments
for i in range(0, num_fragments):
    if i == hardware_index:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = [11,8,16,14],
                                     backend = device, shots = shots // total_variants, 
                                     optimization_level = 3, monitor_jobs = True)
    else:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = None,
                                     backend = simulation_backend, shots = shots // total_variants,
                                     optimization_level = 3, monitor_jobs = True, noise_model = None)
       

job_id:  639189c6004b54c90653383a
Job Status: job has successfully run
job_id:  38c2adb8-34e3-4478-8577-8ad6a0145f60
Job Status: job has successfully run


In [79]:
frag_datas[0]

[Result(backend_name='ibm_auckland', backend_version='1.3.0', qobj_id='60a7e3e2-9fe1-4720-9e5e-deefdf2f0028', job_id='639189c6004b54c90653383a', success=True, results=[ExperimentResult(shots=10000, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 34, '0x1': 30, '0x2': 33, '0x3': 41, '0x4': 22, '0x5': 33, '0x6': 17, '0x7': 20, '0x8': 1471, '0x9': 1524, '0xa': 1577, '0xb': 1560, '0xc': 851, '0xd': 965, '0xe': 882, '0xf': 940}), header=QobjExperimentHeader(clbit_labels=[['c0', 0], ['c0', 1], ['c0', 2], ['c0', 3]], creg_sizes=[['c0', 4]], global_phase=0.0, memory_slots=4, metadata={}, n_qubits=27, name="(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z'))", qreg_sizes=[['q', 27]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4], ['q', 5], ['q', 6], ['q', 7], ['q', 8], ['q', 9], ['q', 10], ['q', 11], ['q', 12], ['q', 13], ['q', 14], ['q', 15], ['q', 16], ['q', 17], ['q', 18], ['q', 19], ['q', 20], ['q', 21], ['q', 22], ['q', 23], ['q', 24], ['q', 25], ['q', 26

In [80]:
# from qiskit.providers.ibmq.managed import IBMQJobManager
# job_manager = IBMQJobManager()

In [81]:
# job_set = job_manager.retrieve_job_set(job_set_id='973c2fdceaf547389a770d19aad463d8-1669966038684177', provider=provider)

In [82]:
# managed_results = job_set.results()

In [104]:
frag_datas1_copy = frag_datas[1].copy()

In [105]:
frag_datas[1] = [frag_datas[1][0].combine_results()]

In [106]:
# frag_datas[0] = run_circuits(multi_exp_circuits[0], initial_layout = None,
#                              backend = simulation_backend, shots = shots // total_variants,
#                              optimization_level = 3, monitor_jobs = True, noise_model = None)

In [84]:
frag_datas[0][0].results[0].header.name

"(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z'))"

In [85]:
num_tomo_circuits

25

In [86]:
test = [[], []]

In [87]:
results_list = [[], []]
counts_list = [[], []] 
for i in range(0, num_fragments): 
    for j in range(0, num_tomo_circuits):
        start_idx = j * 12
        end_idx = (j + 1) * 12
        results_list[i].append(frag_datas[i][0].results[start_idx : end_idx])
        counts_list[i].append(frag_datas[i][0].get_counts()[start_idx : end_idx])


In [88]:
frag_datas[1][0].get_counts()

[{'110': 4930, '010': 5070},
 {'111': 2419, '010': 2609, '110': 2487, '011': 2485},
 {'100': 1, '111': 2393, '110': 2512, '010': 2547, '011': 2547},
 {'101': 1,
  '011': 1,
  '111': 1,
  '100': 3394,
  '110': 1678,
  '010': 1619,
  '000': 3306},
 {'011': 825,
  '111': 834,
  '100': 1672,
  '110': 869,
  '001': 1627,
  '010': 883,
  '000': 1644,
  '101': 1646},
 {'000': 1631,
  '010': 865,
  '111': 812,
  '011': 815,
  '001': 1657,
  '100': 1737,
  '110': 823,
  '101': 1660},
 {'001': 4, '100': 3472, '110': 1632, '010': 1597, '000': 3295},
 {'011': 773,
  '111': 853,
  '001': 1617,
  '110': 811,
  '100': 1785,
  '010': 830,
  '000': 1647,
  '101': 1684},
 {'011': 872,
  '111': 806,
  '110': 761,
  '001': 1614,
  '100': 1725,
  '000': 1664,
  '010': 832,
  '101': 1726},
 {'111': 1, '100': 3331, '110': 1711, '010': 1698, '000': 3259},
 {'110': 892,
  '001': 1569,
  '100': 1717,
  '111': 797,
  '011': 811,
  '010': 848,
  '000': 1712,
  '101': 1654},
 {'110': 835,
  '100': 1684,
  '001': 1

In [89]:
frag_datas[0][0].get_counts()

[{'0000': 34,
  '0001': 30,
  '0010': 33,
  '0011': 41,
  '0100': 22,
  '0101': 33,
  '0110': 17,
  '0111': 20,
  '1000': 1471,
  '1001': 1524,
  '1010': 1577,
  '1011': 1560,
  '1100': 851,
  '1101': 965,
  '1110': 882,
  '1111': 940},
 {'0000': 700,
  '0001': 787,
  '0010': 706,
  '0011': 800,
  '0100': 463,
  '0101': 502,
  '0110': 482,
  '0111': 502,
  '1000': 741,
  '1001': 818,
  '1010': 759,
  '1011': 812,
  '1100': 434,
  '1101': 488,
  '1110': 503,
  '1111': 503},
 {'0000': 730,
  '0001': 774,
  '0010': 750,
  '0011': 855,
  '0100': 454,
  '0101': 504,
  '0110': 467,
  '0111': 487,
  '1000': 785,
  '1001': 761,
  '1010': 741,
  '1011': 783,
  '1100': 457,
  '1101': 478,
  '1110': 472,
  '1111': 502},
 {'0000': 899,
  '0001': 989,
  '0010': 805,
  '0011': 948,
  '0100': 765,
  '0101': 856,
  '0110': 744,
  '0111': 818,
  '1000': 518,
  '1001': 500,
  '1010': 474,
  '1011': 495,
  '1100': 298,
  '1101': 299,
  '1110': 293,
  '1111': 299},
 {'0000': 727,
  '0001': 632,
  '0010': 

In [90]:
from mlrecon_methods import organize_tomography_data_from_list
organized_data = []
for i in range(0, num_tomo_circuits):
    frag_data = []
    frag_targets = ml.identify_frag_targets(wire_path_map_list[i])
    for j in range(0, num_fragments):
        frag_data.append(organize_tomography_data_from_list(results_list[j][i], counts_list[j][i],
                                              frag_targets[j].get("prep"),
                                              frag_targets[j].get("meas"),
                                              prep_basis = "SIC", extra_qc = []))
    organized_data.append(frag_data)

(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z')) ('S0',) ('Z',) [3] [3]
(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'X')) ('S0', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'X')
(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'X')) ('S0',) ('X',) [3] [3]
(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Y')) ('S0', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'Y')
(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Y')) ('S0',) ('Y',) [3] [3]
(('S0', 'S0', 'S0', 'S1'), ('Z', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S1') ('Z', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S1'), ('Z', 'Z', 'Z', 'Z')) ('S1',) ('Z',) [3] [3]
(('S0', 'S0', 'S0', 'S1'), ('Z', 'Z', 'Z', 'X')) ('S0', 'S0', 'S0', 'S1') ('Z', 'Z', 'Z', 'X')
(('S0', 'S0', 'S0', 'S1'), ('Z', 'Z', 'Z', 'X')) ('S1',) ('X',) [3] [3]
(('S0', 'S0', 'S0', 'S1'), ('Z', 'Z', 'Z', 'Y')) ('S0', 'S0', 'S0', 'S1') ('Z', 'Z', 'Z', 'Y')
(('S0', 'S0', 'S0', 'S1'), ('Z', 'Z', 'Z', 'Y')) ('S1',) ('Y',) [3] [3

In [91]:
for final_bits, fixed_bit_data in frag_data[0].items():
    prep_meas_states, state_counts = zip(*fixed_bit_data.items())
    prep_labels, meas_labels = zip(*prep_meas_states)
    prep_qubit_num = len(prep_labels[0])
    meas_qubit_num = len(meas_labels[0])
    print(prep_qubit_num)

1
1
1
1
1
1
1


In [92]:
len(organized_data)

25

In [93]:
organized_data[2]

[{'000': {(('S0',), ('Zp',)): 45,
   (('S0',), ('Zm',)): 2747,
   (('S0',), ('Xp',)): 1369,
   (('S0',), ('Xm',)): 1370,
   (('S0',), ('Yp',)): 1361,
   (('S0',), ('Ym',)): 1374,
   (('S1',), ('Zp',)): 1997,
   (('S1',), ('Zm',)): 932,
   (('S1',), ('Xp',)): 1382,
   (('S1',), ('Xm',)): 1508,
   (('S1',), ('Yp',)): 153,
   (('S1',), ('Ym',)): 2677,
   (('S2',), ('Zp',)): 1986,
   (('S2',), ('Zm',)): 867,
   (('S2',), ('Xp',)): 2587,
   (('S2',), ('Xm',)): 384,
   (('S2',), ('Yp',)): 2082,
   (('S2',), ('Ym',)): 897,
   (('S3',), ('Zp',)): 1890,
   (('S3',), ('Zm',)): 999,
   (('S3',), ('Xp',)): 376,
   (('S3',), ('Xm',)): 2552,
   (('S3',), ('Yp',)): 2206,
   (('S3',), ('Ym',)): 730},
  '001': {(('S0',), ('Zp',)): 48,
   (('S0',), ('Zm',)): 3258,
   (('S0',), ('Xp',)): 1669,
   (('S0',), ('Xm',)): 1688,
   (('S0',), ('Yp',)): 1716,
   (('S0',), ('Ym',)): 1663,
   (('S1',), ('Zp',)): 1570,
   (('S1',), ('Zm',)): 1093,
   (('S1',), ('Xp',)): 1301,
   (('S1',), ('Xm',)): 1353,
   (('S1',)

In [94]:
organized_data[-1][-1]

{'10': {(('S0',), ('Zp',)): 1,
  (('S1',), ('Zp',)): 6687,
  (('S1',), ('Xp',)): 3309,
  (('S1',), ('Xm',)): 3296,
  (('S1',), ('Yp',)): 3270,
  (('S1',), ('Ym',)): 3356,
  (('S2',), ('Zp',)): 6704,
  (('S2',), ('Xm',)): 3330,
  (('S2',), ('Xp',)): 3416,
  (('S2',), ('Yp',)): 3417,
  (('S2',), ('Ym',)): 3354,
  (('S3',), ('Zp',)): 6698,
  (('S3',), ('Xm',)): 3284,
  (('S3',), ('Xp',)): 3337,
  (('S3',), ('Yp',)): 3420,
  (('S3',), ('Ym',)): 3278,
  (('S0',), ('Zm',)): 0,
  (('S0',), ('Xp',)): 0,
  (('S0',), ('Xm',)): 0,
  (('S0',), ('Yp',)): 0,
  (('S0',), ('Ym',)): 0,
  (('S1',), ('Zm',)): 0,
  (('S2',), ('Zm',)): 0,
  (('S3',), ('Zm',)): 0},
 '11': {(('S0',), ('Zp',)): 1,
  (('S0',), ('Xm',)): 1,
  (('S0',), ('Xp',)): 1,
  (('S1',), ('Yp',)): 1,
  (('S2',), ('Zp',)): 1,
  (('S2',), ('Ym',)): 3,
  (('S2',), ('Yp',)): 1,
  (('S0',), ('Zm',)): 0,
  (('S0',), ('Yp',)): 0,
  (('S0',), ('Ym',)): 0,
  (('S1',), ('Zp',)): 0,
  (('S1',), ('Zm',)): 0,
  (('S1',), ('Xp',)): 0,
  (('S1',), ('Xm'

In [95]:
recombined_lists = []
filtered_recombined_lists = []
norm_filtered_lists = []
for i in range(0, num_tomo_circuits):
    direct_models = ml.direct_fragment_model(organized_data[i])
    likely_models = ml.maximum_likelihood_model(direct_models)

    direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map_list[i])
    likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map_list[i])
    recombined_lists.append(likely_recombined_dist)
    filtered_recombined_lists.append(filter_results(likely_recombined_dist, [0]))
    norm_filtered_lists.append(norm_dict(filter_results(likely_recombined_dist, [0])))

{'000': {(('S0',), ('Zp',)): 34, (('S0',), ('Zm',)): 1471, (('S0',), ('Xp',)): 700, (('S0',), ('Xm',)): 741, (('S0',), ('Yp',)): 730, (('S0',), ('Ym',)): 785, (('S1',), ('Zp',)): 899, (('S1',), ('Zm',)): 518, (('S1',), ('Xp',)): 727, (('S1',), ('Xm',)): 613, (('S1',), ('Yp',)): 71, (('S1',), ('Ym',)): 1354, (('S2',), ('Zp',)): 912, (('S2',), ('Zm',)): 470, (('S2',), ('Xp',)): 1237, (('S2',), ('Xm',)): 172, (('S2',), ('Yp',)): 982, (('S2',), ('Ym',)): 374, (('S3',), ('Zp',)): 827, (('S3',), ('Zm',)): 548, (('S3',), ('Xp',)): 124, (('S3',), ('Xm',)): 1305, (('S3',), ('Yp',)): 1012, (('S3',), ('Ym',)): 392}, '001': {(('S0',), ('Zp',)): 30, (('S0',), ('Zm',)): 1524, (('S0',), ('Xp',)): 787, (('S0',), ('Xm',)): 818, (('S0',), ('Yp',)): 774, (('S0',), ('Ym',)): 761, (('S1',), ('Zp',)): 989, (('S1',), ('Zm',)): 500, (('S1',), ('Xp',)): 632, (('S1',), ('Xm',)): 817, (('S1',), ('Yp',)): 100, (('S1',), ('Ym',)): 1321, (('S2',), ('Zp',)): 1033, (('S2',), ('Zm',)): 534, (('S2',), ('Xp',)): 1335, (

{'000': {(('S0',), ('Zp',)): 2, (('S0',), ('Zm',)): 92, (('S0',), ('Xp',)): 34, (('S0',), ('Xm',)): 35, (('S0',), ('Yp',)): 29, (('S0',), ('Ym',)): 30, (('S1',), ('Zp',)): 29, (('S1',), ('Zm',)): 23, (('S1',), ('Xp',)): 18, (('S1',), ('Xm',)): 38, (('S1',), ('Yp',)): 19, (('S1',), ('Ym',)): 29, (('S2',), ('Zp',)): 20, (('S2',), ('Zm',)): 29, (('S2',), ('Xp',)): 28, (('S2',), ('Xm',)): 23, (('S2',), ('Yp',)): 30, (('S2',), ('Ym',)): 20, (('S3',), ('Zp',)): 16, (('S3',), ('Zm',)): 30, (('S3',), ('Xp',)): 13, (('S3',), ('Xm',)): 31, (('S3',), ('Yp',)): 37, (('S3',), ('Ym',)): 21}, '001': {(('S0',), ('Zp',)): 1, (('S0',), ('Zm',)): 69, (('S0',), ('Xp',)): 45, (('S0',), ('Xm',)): 32, (('S0',), ('Yp',)): 33, (('S0',), ('Ym',)): 47, (('S1',), ('Zp',)): 26, (('S1',), ('Zm',)): 28, (('S1',), ('Xp',)): 28, (('S1',), ('Xm',)): 27, (('S1',), ('Yp',)): 23, (('S1',), ('Ym',)): 36, (('S2',), ('Zp',)): 34, (('S2',), ('Zm',)): 39, (('S2',), ('Xp',)): 35, (('S2',), ('Xm',)): 23, (('S2',), ('Yp',)): 28, 

{'000': {(('S0',), ('Zp',)): 55, (('S0',), ('Zm',)): 2543, (('S0',), ('Xp',)): 1360, (('S0',), ('Xm',)): 1313, (('S0',), ('Yp',)): 1276, (('S0',), ('Ym',)): 1343, (('S1',), ('Zp',)): 1715, (('S1',), ('Zm',)): 880, (('S1',), ('Xp',)): 1101, (('S1',), ('Xm',)): 1561, (('S1',), ('Yp',)): 159, (('S1',), ('Ym',)): 2479, (('S2',), ('Zp',)): 1700, (('S2',), ('Zm',)): 925, (('S2',), ('Xp',)): 2430, (('S2',), ('Xm',)): 192, (('S2',), ('Yp',)): 1635, (('S2',), ('Ym',)): 969, (('S3',), ('Zp',)): 1706, (('S3',), ('Zm',)): 893, (('S3',), ('Xp',)): 475, (('S3',), ('Xm',)): 2180, (('S3',), ('Yp',)): 2079, (('S3',), ('Ym',)): 572}, '001': {(('S0',), ('Zp',)): 1, (('S0',), ('Zm',)): 20, (('S0',), ('Xp',)): 12, (('S0',), ('Xm',)): 11, (('S0',), ('Yp',)): 10, (('S0',), ('Ym',)): 10, (('S1',), ('Zp',)): 24, (('S1',), ('Zm',)): 5, (('S1',), ('Xp',)): 17, (('S1',), ('Xm',)): 12, (('S1',), ('Yp',)): 19, (('S1',), ('Ym',)): 7, (('S2',), ('Zp',)): 22, (('S2',), ('Zm',)): 9, (('S2',), ('Xp',)): 20, (('S2',), ('

{'01': {(('S0',), ('Zp',)): 10000, (('S0',), ('Xp',)): 5118, (('S0',), ('Xm',)): 4880, (('S0',), ('Yp',)): 5052, (('S0',), ('Ym',)): 4946, (('S1',), ('Zp',)): 3344, (('S1',), ('Xp',)): 1786, (('S1',), ('Xm',)): 1590, (('S1',), ('Ym',)): 1665, (('S1',), ('Yp',)): 1629, (('S2',), ('Zp',)): 3301, (('S2',), ('Xm',)): 1586, (('S2',), ('Xp',)): 1645, (('S2',), ('Yp',)): 1686, (('S2',), ('Ym',)): 1586, (('S3',), ('Zm',)): 1, (('S3',), ('Zp',)): 3293, (('S3',), ('Xm',)): 1655, (('S3',), ('Xp',)): 1714, (('S3',), ('Yp',)): 1663, (('S3',), ('Ym',)): 1743, (('S0',), ('Zm',)): 0, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0}, '10': {(('S0',), ('Xp',)): 2, (('S1',), ('Zp',)): 6654, (('S1',), ('Xp',)): 3344, (('S1',), ('Xm',)): 3279, (('S1',), ('Ym',)): 3373, (('S1',), ('Yp',)): 3333, (('S2',), ('Zp',)): 6696, (('S2',), ('Xp',)): 3376, (('S2',), ('Xm',)): 3393, (('S2',), ('Yp',)): 3381, (('S2',), ('Ym',)): 3346, (('S3',), ('Zp',)): 6706, (('S3',), ('Xm',)): 3230, (('S3',), ('Xp',)): 3400, (('S3',), 

In [96]:
# direct_models = ml.direct_fragment_model(frag_data)
# likely_models = ml.maximum_likelihood_model(direct_models)

# direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map)
# likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map)

In [97]:
direct_recombined_dist

{'00010': 0.029333784969449182,
 '10010': 5.744114832271037e-07,
 '10000': 0.0008495539578407008,
 '00000': 6.397398566158556e-07,
 '00011': 0.9440228248821912,
 '10011': -9.294090409139715e-05,
 '10001': 0.001033838775704285,
 '00001': 8.639872927715495e-06,
 '00111': 0.010728604800310147,
 '10111': 1.3068638693260508e-06,
 '10101': 0.0007359465591970023,
 '00101': 2.2904114022660344e-07,
 '01011': 0.013913564266544096,
 '11011': -1.1486705005411353e-06,
 '11001': -0.0017721556511346138,
 '01001': 3.291537744307357e-07,
 '00110': 0.0005354182153688242,
 '10110': 3.612384471261957e-08,
 '10100': -6.848680044743036e-05,
 '00100': 2.711843791977393e-08,
 '01010': 0.0006746904259095932,
 '11010': 1.462063208202014e-08,
 '11000': -4.119096825297764e-05,
 '01000': 1.6006020834784322e-08,
 '01111': 0.0001673207516835779,
 '11111': -3.257759167850833e-08,
 '11101': -3.1424665800683496e-05,
 '01101': 1.9681633644068464e-08}

In [98]:
likely_recombined_dist

{'00010': 0.028397939623443808,
 '10000': 0.001419417207587142,
 '00011': 0.9224365365332944,
 '10001': 0.02286859997578435,
 '00111': 0.010209248993193662,
 '10101': 0.0005858916588910721,
 '01011': 0.013334047930660257,
 '11001': 5.5430629248374885e-05,
 '00110': 0.00025703579616636116,
 '10100': 8.59446647049456e-06,
 '01010': 0.0004028901928639235,
 '11000': 1.7588356622713658e-05,
 '01111': 5.411831348175993e-06,
 '11101': 1.366804425283758e-06}

In [99]:
filtered_recombined_lists

[{'0010': 0.07445616934525817,
  '0000': 0.07207506915351528,
  '0110': 0.07752015458571745,
  '0100': 0.07507305150910855,
  '1010': 0.07756673828558516,
  '1000': 0.07506662854197574,
  '1110': 0.07942172474642031,
  '1100': 0.0769391605523546,
  '0011': 0.043897827798184526,
  '0001': 0.0425241344816832,
  '0111': 0.04850140396313997,
  '0101': 0.0470116023823029,
  '1011': 0.04553646491149315,
  '1001': 0.04409882080503919,
  '1111': 0.04809209537826358,
  '1101': 0.04659890418236357},
 {'0010': 0.13037610571494193,
  '0110': 0.13271388714127105,
  '1010': 0.12774767031983392,
  '1110': 0.13673702375712116,
  '0011': 0.1102480431216984,
  '0111': 0.11022342152403636,
  '1011': 0.10844890238215522,
  '1111': 0.11813000668368419},
 {'0010': 0.13500458713715055,
  '0000': 0.13605442452002,
  '0110': 0.16159704602719463,
  '0100': 0.16252958318845329,
  '1110': 0.0013627731743617606,
  '1100': 0.001372752576051311,
  '0011': 0.08353109232997206,
  '0001': 0.08448115890556242,
  '0111':

In [100]:
from utils.utils import filter_results

In [101]:
filter_direct_recombined = filter_results(direct_recombined_dist, [0])

In [102]:
filter_direct_recombined

{'0010': 0.029333784969449182,
 '0000': 6.397398566158556e-07,
 '0011': 0.9440228248821912,
 '0001': 8.639872927715495e-06,
 '0111': 0.010728604800310147,
 '0101': 2.2904114022660344e-07,
 '1011': 0.013913564266544096,
 '1001': 3.291537744307357e-07,
 '0110': 0.0005354182153688242,
 '0100': 2.711843791977393e-08,
 '1010': 0.0006746904259095932,
 '1000': 1.6006020834784322e-08,
 '1111': 0.0001673207516835779,
 '1101': 1.9681633644068464e-08}

In [103]:

filter_likely_recombined = filter_results(likely_recombined_dist, [0])

In [104]:
filter_likely_recombined

{'0010': 0.028397939623443808,
 '0011': 0.9224365365332944,
 '0111': 0.010209248993193662,
 '1011': 0.013334047930660257,
 '0110': 0.00025703579616636116,
 '1010': 0.0004028901928639235,
 '1111': 5.411831348175993e-06}

In [105]:
norm_filter_dist = norm_dict(filter_likely_recombined)

In [106]:
norm_filter_dist

{'0010': 0.02912480413015094,
 '0011': 0.9460469247159071,
 '0111': 0.0104705616388182,
 '1011': 0.01367534192240913,
 '0110': 0.00026361480153308504,
 '1010': 0.0004132024403430124,
 '1111': 5.55035083851348e-06}

In [107]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(norm_filtered_lists[i], shots = total_counts(norm_filtered_lists[i]), Pauli = Pauli_tuple[0])  

In [108]:
final_expect_val

(-1.0648051426111258+0j)

In [109]:
import json
with open('LiH_auckland_check1.json', 'w') as f:
    json.dump(norm_filtered_lists, f)

In [110]:
mit = mthree.M3Mitigation(device)

In [111]:
mit.cals_from_file(cals_file = 'mit_data')

In [112]:
def update_dict(quasis, total_shots = None):
    for key in quasis.keys():
        quasis[key] = quasis[key]*total_shots

In [113]:
mit_lists = []
for i in range(0, len(pauli_commute)):
    total_shots = total_counts(norm_filtered_lists[i])
    quasis = mit.apply_correction(norm_filtered_lists[i], [16,14,11,8])
    update_dict(quasis, total_shots = total_shots)
    mit_lists.append(quasis)

In [114]:
mit_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        mit_val += coeff * evaluation(mit_lists[i], shots = total_counts(mit_lists[i]), Pauli = Pauli_tuple[0])  

In [115]:
mit_val

(-1.0798425593958028+0j)

In [116]:
with open('LiH_auckland_check1_mit.json', 'w') as f:
    json.dump(mit_lists, f)